In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd  
import requests 
import  numpy as np
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.auto import tqdm


/media/rdi/043764CF611486F91/NLP/llm-zoomcamp/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Q1. Getting the embeddings model

0.07



In [2]:

model_name="multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)
user_question = "I just discovered the course. Can I still join it?"
user_question_embedding = embedding_model.encode(user_question)


In [35]:
print(user_question_embedding[0])

0.07822265


#### Prepare the documents


In [5]:

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
print(documents[0].keys())
# Filter documents for "machine-learning-zoomcamp"
filtered_documents = [
    doc for doc in documents if doc['course'].startswith("machine-learning-zoomcamp")
]
len(filtered_documents)

dict_keys(['text', 'section', 'question', 'course', 'id'])


375

### Q2. Creating the embeddings
(375, 768)


In [6]:
embeddings = [
    embedding_model.encode(
        '{question} {text}'.format(**doc)
    ) for doc in tqdm(filtered_documents)
]
# Convert to numpy array
X = np.array(embeddings)
# Print the shape of X
print(X.shape)

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [02:16<00:00,  2.74it/s]

(375, 768)


### Q3. Search
0.6506573


In [7]:
scores = X.dot(user_question_embedding)
max(scores)

0.6506573

#### Vector search


In [8]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]



In [9]:
search_engine = VectorSearchEngine(
    documents=filtered_documents, embeddings=X
)
search_engine.search(
    user_question_embedding, num_results=5
)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine
0.93

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [34]:
def relevance_total(ground_truth, search_function):
    for q in tqdm(ground_truth):
        doc_id = q['document']
        question_vector = embedding_model.encode(q['question'])
        results = search_function(question_vector)
        relevance = [d['id'] == doc_id for d in results]
        yield relevance

In [12]:
relevance_total_ours = list(
    relevance_total(
        ground_truth,
        lambda q : search_engine.search(
            q, num_results=5
        )
    )
)


100%|██████████| 1830/1830 [04:46<00:00,  6.39it/s]


In [13]:
def hit_rate(relevance_total):
    return sum(
        True in line for line in relevance_total
    ) / len(relevance_total)
hit_rate(relevance_total_ours)


0.9398907103825137

### Q5. Indexing with Elasticsearch
ee58a693

In [25]:
# Connect to Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()


ObjectApiResponse({'name': '553b54310d14', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1niU0FATTbyLwEw_mOn7Rw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "qa_vector": {"type": "dense_vector",
                            "dims": 768,
                            "index": True,
                            "similarity": "cosine"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)
# Index the filtered_documents
print(filtered_documents[0])

for doc, vec in tqdm(zip(filtered_documents, embeddings)):
    doc['qa_vector'] = vec

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I sign up?', 'course': 'machine-learning-zoomcamp', 'id': '0227b872', 'qa_vector': array([ 8.80590379e-02,  1.55935558e-02,  7.92557821e-02,  2.52758171e-02,
        7.55765140e-02, -3.90597060e-02, -4.13814224e-02,  2.52918322e-02,
        2.43241359e-02,  3.62586160e-03, -7.28285639e-03, -3.28751393e-02,
        6.12956136e-02, -5.71100675e-02,  1.16774738e-02, -1.79441944e-02,
        4.49206010e-02, -5.41606024e-02, -1.92263431e-03,  1.48330284e-02,
        7.91357830e-03, -3.43127325e-02,  1.21480543e-02,  

375it [00:00, 731564.65it/s]


In [27]:
for doc in tqdm(filtered_documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|██████████| 375/375 [00:06<00:00, 55.73it/s]


In [28]:
def elastic_search(field, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": { "term": { "course": course }}
    }

    search_query = {
        "knn": knn_query,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name, body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [29]:
elastic_search('qa_vector', user_question_embedding, 'machine-learning-zoomcamp')


[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

### Q6. Hit-rate for Elasticsearch

0.93


In [30]:
def es_search_function(vector):
    return elastic_search(
        'qa_vector',
        user_question_embedding[0],
        'machine-learning-zoomcamp'
    )

relevance_total_es = list(
    relevance_total(ground_truth,es_search_function)
)


100%|██████████| 1830/1830 [03:50<00:00,  7.93it/s]


In [33]:
hit_rate(relevance_total_es)


0.01366120218579235